In [1]:
import pandas as pd
import numpy as np
import codecs
from tqdm import tqdm
import re
from datetime import datetime

In [7]:
df = pd.read_csv("data/南海自由航行_news_newlabel.csv")

In [25]:
event_info = {}
eventtoid = {}
idtoevent = {}
id_num = 0
with codecs.open("dict/事件节点.txt","r","UTF-8") as rf:
    for line in rf.readlines():
        event_center, event_des = line.strip().split(":")
        event_info[event_center] = event_des
        eventtoid[event_center] = id_num
        idtoevent[id_num] = event_center
        id_num += 1
eventtoid

{'入侵行动': 0, '防卫行动': 1, '访问行动': 2, '军演行动': 3, '国家立场': 4, '媒体评论': 5}

In [26]:
# 利用二维列表表示关系图
event_graph = np.zeros([id_num, id_num])
event_graph

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

# 根据新闻标题的内容共现关系构建事件中心关系图

- 根据标题的内容标签共现构建关系图

In [27]:
# 依据新闻标题构建事件中心关系图
for content_label_info in tqdm(df['content_label']):
    labels = content_label_info.split(' ')
    # print(labels)
    for i in range(len(labels)-1):
        for j in range(i+1, len(labels)):
            event_graph[eventtoid[labels[i]]][eventtoid[labels[j]]] += 1
            event_graph[eventtoid[labels[j]]][eventtoid[labels[i]]] += 1
event_graph

100%|██████████████████████████████████████████████████████████████████████████| 4943/4943 [00:00<00:00, 411913.79it/s]


array([[  0., 152.,   1.,  42., 290., 148.],
       [152.,   0.,   1.,   8.,  99.,  46.],
       [  1.,   1.,   0.,   0.,   3.,   6.],
       [ 42.,   8.,   0.,   0.,  28.,  34.],
       [290.,  99.,   3.,  28.,   0.,  81.],
       [148.,  46.,   6.,  34.,  81.,   0.]])

# 根据新闻内容构建事件中心关系图

- 根据文章共现关系构建关系图,分句-分类-构建
- 根据文章前后句连接词构建关系图 分句-分类-识别连接词-构建

In [14]:
# 依据新闻内容构建事件中心关系图
iter_num = 0
for content in df['content']:
    # content = ''.join(content.split()) #split方法中不带参数时，表示分割所有换行符、制表符、空格
    # content = content.replace(u'\u3000', u'').replace('\n','').replace(' ','').replace('\r','')
    # content_words = re.split('。|！|？',content)
    content_words = content.split()
    # print(content_words)
    if iter_num > 10: break
    iter_num += 1

['原标题：美方在南海开展“航行自由行动”？中方：人不犯我', '我不犯人']
['外交部：坚决反对打着航行自由幌子损害南海沿岸国主权与安全利益的行为', '外交部发言人华春莹16日在例行记者会上表示，中国坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '在当日的记者会上，有记者问：美国副总统彭斯说南海不属于任何一个国家，美国将继续在国际法允许的范围内行使航行和飞越自由。中方是否认为彭斯的相关言论是种挑衅？', '华春莹说，南海航行和飞越自由从来不存在任何问题。“我们曾经多次问美方或者声称南海航行自由有问题的人，请拿出证据来，证明有哪个国家的哪艘船、哪架飞机、什么时候在南海航行或者飞越时遇到了问题？但到目前为止，没有任何人给出一个明确的答案。由此可见，所谓南海航行与飞越自由根本就是一个不存在的伪命题。”', '华春莹表示，中方的立场非常明确，愿与地区国家一道，坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对那些打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '“另外，请你提醒一下美方有关人士，美国到现在还没有批准《联合国海洋法公约》。如果美方能够尽早批准并遵守《公约》，我想将更加有利于维护南海地区的和平稳定。”华春莹说。', '消息来源：新华社', '本期编审：孙', '利', '责任编辑：朱西迪、李明', '投稿邮箱：gfsk@cnr.cn', '转载请注明出处！']
['一艘英国皇家海军军舰日前驶向越南途中，在接近中国于南海声索的岛屿附近航行，以彰显“航行自由”，但遭中国方面派出舰船及直升机驱离。']
['外交部发言人华春莹16日在例行记者会上表示，中国坚定维护各国依据国际法所享有的真正的航行与飞越自由，同时坚决反对打着所谓航行自由的幌子损害南海沿岸国主权与安全利益的行为。', '在当日的记者会上，有记者问：美国副总统彭斯说南海不属于任何一个国家，美国将继续在国际法允许的范围内行使航行和飞越自由。中方是否认为彭斯的相关言论是种挑衅？', '华春莹说，南海航行和飞越自由从来不存在任何问题。“我们曾经多次问美方或者声称南海航行自由有问题的人，请拿出证据来，证明有哪个国家的哪艘船、哪架飞机、什么时候在南海航行或者飞越时遇到了问题？但到目前为止，没有任何人给出

# 效果可视化

In [34]:
from pyecharts import options as opts
from pyecharts.charts import Graph


nodes_data = [opts.GraphNode(name=event_info[event_key], symbol_size=40) for event_key in eventtoid.keys()]
links_data = []

[rows, cols] = event_graph.shape
for i in range(rows):
    for j in range(cols):
        if i == j: continue
        if event_graph[i, j] == 0: continue
        links_data.append(opts.GraphLink(source=event_info[idtoevent[i]], target=event_info[idtoevent[j]], value=event_graph[i, j]))

graph = (
    Graph()
    .add(
        "",
        nodes_data,
        links_data,
        repulsion=4000,
        edge_label=opts.LabelOpts(
            is_show=True, position="middle", formatter="{c}"
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="南海事件")
    )
    # .render("result/event_center_graph.html")
)
graph.render_notebook()

# 事件-时间演化图构建

In [3]:
df = pd.read_csv("data/南海自由航行_news_newlabel.csv")

In [4]:
df['time'] = pd.to_datetime(df['time'])
df['time'] = [datetime.strftime(x,'%Y-%m-%d') for x in df['time']]
# df = df.set_index('time')
df = df[(df['time'] >= '2019-11-20') & (df['time'] <= '2019-11-27')] # 筛选时间段
df['time']

69      2019-11-22
194     2019-11-23
344     2019-11-20
384     2019-11-23
450     2019-11-27
           ...    
4796    2019-11-25
4850    2019-11-27
4851    2019-11-27
4853    2019-11-27
4854    2019-11-27
Name: time, Length: 119, dtype: object

In [5]:
group_result = df.groupby (df['time'])

In [12]:
from cluster_vps import k_means_tfidf
import codecs
import json
json_result = {}  # 输出为json文件反馈给前端
with codecs.open("result/news_date_k_means_clusters.txt", "w", "utf-8") as wf:
    for time, day_df in group_result:
        print(time)
        json_result[time] = []
        
        title_list = day_df['title'].tolist()
        result = k_means_tfidf(title_list, 1, 5)
        wf.write(time + "\n")
        for key in result[0].keys():
            tmp = {}
            wf.write("cluster" + str(key) + "\n")
            wf.write("center: " + result[1][key] + "\n")
            tmp['cluster'] = str(key)
            tmp['center'] = result[1][key]
            tmp['title_list'] = []
            for i in result[0][key]:
                wf.write(title_list[i] + "\n")
                tmp['title_list'].append(title_list[i])
            wf.write("\n")
            json_result[time].append(tmp)
        wf.write("=============\n\n")
with codecs.open("result/20191120-20191127_titlecluster.json", "w", 'utf-8') as wf:
    json.dump(json_result, wf, indent=4)

2019-11-20
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.03494684619278409
2019-11-22
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.24097830897037528
2019-11-23
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.30804076463107805
2019-11-24
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.3288123460202369
2019-11-25
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.003755232233632305
2019-11-26
2019-11-27
Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)
3 0.35822082352291645
